In [1]:
#%
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tqdm
import cx_Oracle


In [2]:
cx_Oracle.init_oracle_client(lib_dir=r"C:/Oracle/instantclient_19_19")

In [3]:
#%
connection = cx_Oracle.connect(user='scott', password='tiger', dsn='192.168.119.119:1521/dink11.dbsvr')
cursor = connection.cursor()
sql="select categorySmall, convenienceToEmbedding from CATEGORYembedding"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['categorySmall','convenienceToEmbedding']
category = pd.DataFrame(rows, columns=columns)
# category



In [4]:
sql="SELECT c.email as email,ch.categorySmall, ch.amount FROM cardhistory ch JOIN card c ON ch.cardid = c.cardid"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['email','categorySmall','amount']
data = pd.DataFrame(rows, columns=columns)
data

,email,categorySmall,amount
0,gangjimin@live.com,일식/중식/양식,46000
1,gangjimin@live.com,제과점,2500
2,gangjimin@live.com,편의점,22700
3,gangjimin@live.com,편의점,16300
4,gangjimin@live.com,제과점,10500
...,...,...,...
199995,eunseo08@hanmail.net,슈퍼마켓,21000
199996,eunseo08@hanmail.net,약국,16700
199997,eunseo08@hanmail.net,의복/의류,88000
199998,eunseo08@hanmail.net,커피전문점,8800


In [5]:
data['categoryEmbedding'] = data['categorySmall'].replace(category.set_index('categorySmall')['convenienceToEmbedding'])
data

,email,categorySmall,amount,categoryEmbedding
0,gangjimin@live.com,일식/중식/양식,46000,0.212529
1,gangjimin@live.com,제과점,2500,0.432291
2,gangjimin@live.com,편의점,22700,1.000000
3,gangjimin@live.com,편의점,16300,1.000000
4,gangjimin@live.com,제과점,10500,0.432291
...,...,...,...,...
199995,eunseo08@hanmail.net,슈퍼마켓,21000,0.501490
199996,eunseo08@hanmail.net,약국,16700,0.391269
199997,eunseo08@hanmail.net,의복/의류,88000,0.125030
199998,eunseo08@hanmail.net,커피전문점,8800,0.439538


In [6]:
userList=data['email'].unique()

In [7]:
data.head()

,email,categorySmall,amount,categoryEmbedding
0,gangjimin@live.com,일식/중식/양식,46000,0.212529
1,gangjimin@live.com,제과점,2500,0.432291
2,gangjimin@live.com,편의점,22700,1.000000
3,gangjimin@live.com,편의점,16300,1.000000
4,gangjimin@live.com,제과점,10500,0.432291


In [8]:
mode_card_df = {'email': [], 'categorySmall' : [], 'embedding_mode' : [],'amount':[]}
for i, userId in tqdm.tqdm(enumerate(userList)):
    '''
    _df : 카드 거래 내역 중 카드번호 같은 것 끼리 모은 임시 df
    mean_card_df : 카드번호 같은 것들 중, 최빈값 업종을 뽑아서 대표 업종으로 만든 df
    '''
    _df = data[data['email']==userId][['categorySmall', 'categoryEmbedding','amount']]
    # print(_df)
    # print(_df['categorySmall'].mode().iloc[0])
    # print(_df['amount'].mean())

    mode_card_df['email'].append(userId)
    mode_card_df['categorySmall'].append(_df['categorySmall'].mode().iloc[0])
    mode_card_df['embedding_mode'].append(_df['categoryEmbedding'].mode().iloc[0])
    mode_card_df['amount'].append(_df['amount'].mean())
    
# mode_card_df = pd.DataFrame(mode_card_df)
# print(mode_card_df)
print(len(mode_card_df['email']))
print(len(mode_card_df['categorySmall']))
print(len(mode_card_df['amount']))
mode_card_df = pd.DataFrame(mode_card_df)
mode_card_df['amount'] = mode_card_df['amount'].astype(int)
mode_card_df

200it [00:02, 72.30it/s]

200
200
200


,email,categorySmall,embedding_mode,amount
0,gangjimin@live.com,한식,0.178922,62578
1,coeminji@dreamwiz.com,한식,0.178922,64833
2,baedohyeon@hotmail.com,편의점,1.000000,72944
3,hyejinbag@hanmail.net,한식,0.178922,61646
4,igim@daum.net,한식,0.178922,68379
...,...,...,...,...
195,minji81@naver.com,한식,0.178922,84944
196,misuggim@dreamwiz.com,한식,0.178922,70390
197,gimgyeongsu@nate.com,한식,0.178922,92952
198,gyeongja17@hanmail.net,한식,0.178922,71376


In [9]:
# mode_card_df['embedding_mode'].values
# mode_card_df['amount'].values

In [10]:
from sklearn.cluster import KMeans

n_clusters = 4
X = np.array(list(zip(mode_card_df['embedding_mode'].values, mode_card_df['amount'].values)))
print(X.shape)

kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
cluster_labels = kmeans.labels_
print(len(cluster_labels))
mode_card_df['Cluster'] = cluster_labels

mode_card_df

(200, 2)
200


,email,categorySmall,embedding_mode,amount,Cluster
0,gangjimin@live.com,한식,0.178922,62578,1
1,coeminji@dreamwiz.com,한식,0.178922,64833,1
2,baedohyeon@hotmail.com,편의점,1.000000,72944,1
3,hyejinbag@hanmail.net,한식,0.178922,61646,2
4,igim@daum.net,한식,0.178922,68379,1
...,...,...,...,...,...
195,minji81@naver.com,한식,0.178922,84944,3
196,misuggim@dreamwiz.com,한식,0.178922,70390,1
197,gimgyeongsu@nate.com,한식,0.178922,92952,0
198,gyeongja17@hanmail.net,한식,0.178922,71376,1


In [11]:
mode_card_df['Cluster'].value_counts()

1    86
0    40
3    38
2    36
Name: Cluster, dtype: int64

In [24]:
mode_card_df['email']
mode_card_df['Cluster']=mode_card_df['Cluster'].astype(str)
mode_card_df['Cluster']

0      1
1      1
2      1
3      2
4      1
      ..
195    3
196    1
197    0
198    1
199    1
Name: Cluster, Length: 200, dtype: object

In [30]:
cursor = connection.cursor()
insertSql="INSERT INTO clusterTable (email,clusterNum) values(:1,:2)"
for i in range(len(mode_card_df)):
    cursor.execute(insertSql, (mode_card_df.loc[i,'email'],mode_card_df.loc[i,'Cluster']))
connection.commit()
cursor.close()
connection.close()  